# Demo code associated with article "Unveiling Inter-Embryo Variability in Spindle Length over time: towards Quantitative Phenotype Analysis." by Y. Le Cunff et al. 2024

Code author: Jacques Pécréaux

May 2024

CNRS, Univ Rennes, IGDR (Institut de Génétique et Développement de Rennes) – UMR 6290, F-35000 Rennes, France

License: [CeCILL v2.1, see file Licence_CeCILL_V2.1-en.txt](./Licence_CeCILL_V2.1-en.txt)

In [25]:
# temperature considered
temp="18C" # "18C", "23C" with the provided dataset

# We will also download the interaction from wormbase
current_WormBase_Release='WS292'

# Logistic regression will return the probability that a gene interacts with another. We need a threshold to classify between interacting and non-interacting
proba_theshold_prediction=0.9

# we also need the PCA score file output by PCA_paper_code.ipynb
pca_scores_file='scores_relative_to_none'+temp+'.csv'

## Imports, (down)loading and subfunctions

In [26]:
import pandas as pd
import json
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, recall_score, precision_score
import requests
import os
import pathlib

In [27]:
# Download table from wormbase for correspondance between gene name and id
url=f"https://downloads.wormbase.org/releases/current-production-release/species/c_elegans/PRJNA13758/annotation/c_elegans.PRJNA13758.{current_WormBase_Release}.geneIDs.txt.gz"
cur_dir=pathlib.Path().resolve()
# Create the Data folder if it doesn't exist
if not os.path.exists(os.path.join(cur_dir,'Data')):
    os.makedirs('Data')
# Define the file path
file_path = 'Data/' + url.split('/')[-1]
# Download the file
response = requests.get(url)
with open(file_path, 'wb') as file:
    file.write(response.content)
print(f"File downloaded successfully to {file_path}")
WBgene = pd.read_csv(file_path, compression='gzip', sep=',', header=None, names=['dummy', 'geneId', 'geneName', 'sequence', 'status', 'type'])

File downloaded successfully to Data/c_elegans.PRJNA13758.WS292.geneIDs.txt.gz


In [28]:
# Load the PCA scores
df = pd.read_csv(pca_scores_file)
display(df)

,gene,principal component 1,principal component 2,principal component 3,tempe,strain
0,ANA019none,-1.043194,1.466452,5.329549,18C,ANA019
1,JEP13gpr1,0.273005,-0.840942,-0.350799,18C,JEP13
2,JEP14gpr2-18C,-1.030865,-3.242924,1.638099,18C,JEP14
3,JEP15L4440,0.231701,-2.249044,4.431642,18C,JEP15
4,JEP1klp13-18C,-0.669484,-2.066330,0.301457,18C,JEP1
...,...,...,...,...,...,...
83,unc59,-0.178511,1.244468,-10.093683,18C,TH27
84,vab8,-0.375866,-0.210577,0.648375,18C,TH27
85,zen4,1.288270,8.995004,-10.353139,18C,TH27
86,zyg11,-0.769377,-9.888831,-0.444174,18C,TH27


In [29]:
def find_interactor(WBgoi,json_file,tempe=temp,df=df,proba_theshold_prediction=proba_theshold_prediction):
    goi=WBgoi.replace('-', '')
    df=df.loc[ df['tempe'] == tempe]
    gene_interacting=list()
    
    # get the interactors of klp-19 from wormbase
    # Assuming the JSON file is named "interactors.json"
    with open(json_file) as file:
        data = json.load(file)
    #display(data)
    for i in data['fields']['interactions']['data']['edges']:
        if not i['effector']['label'] in gene_interacting:
            gene_interacting.append(i['effector']['label'])
        if not i['affected']['label'] in gene_interacting:
            gene_interacting.append(i['affected']['label'])
    #display(gene_interacting)
    gene_interacting_formated = [item.replace('-', '') for item in gene_interacting]
    gene_interacting_formated.append(goi) # append the gene of interest itself
    
    # get interacting genes in our dataset
    cols=df.columns
    selected_cols = [col for col in cols if goi in col]
    if not selected_cols:
        dist = df[['gene', 'principal component 1', 'principal component 2', 'principal component 3' ]].copy()
    else:
        dist_col=selected_cols[0]
        dist = df[['gene', dist_col, 'principal component 1', 'principal component 2', 'principal component 3' ]].copy()
    dist['interacting'] = dist['gene'].apply(lambda x: any(item in x for item in gene_interacting_formated))

    # create the train set using control and non-treated as non interacting
    dist_train=dist.loc[np.logical_or(dist['interacting'], np.logical_or(dist['gene'].map(lambda x:x.startswith('none')),np.asarray(dist['gene'].map(lambda x:x.startswith('L4440')))) )]
    print(f"---------- training set {WBgoi}-------")
    pd.set_option('display.max_rows', 0) 
    display(dist_train)
    pd.set_option('display.max_rows', 10) 
    print(any(dist_train[['interacting']]))
    if not any(dist_train[['interacting']].to_numpy()):
        raise Exception("Something wrong in naming ? No interacting gene in our dataset")
    
    # Logistic regression
    clf = LogisticRegression(random_state=0,solver='liblinear',verbose=1,fit_intercept=False).fit(dist_train[['principal component 1', 'principal component 2', 'principal component 3' ]].to_numpy().reshape(-1, 3), dist_train['interacting'].to_numpy())
    # Predict the labels for the training dataset
    train_predictions = clf.predict(dist_train[['principal component 1', 'principal component 2', 'principal component 3']].to_numpy().reshape(-1, 3))
    # Compute accuracy
    accuracy = accuracy_score(dist_train['interacting'].to_numpy(), train_predictions)
    # Compute recall
    recall = recall_score(dist_train['interacting'].to_numpy(), train_predictions)
    # Compute precision
    precision = precision_score(dist_train['interacting'].to_numpy(), train_predictions)
    print(f"On the training set: Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}")

    # annotate our dataset with predictions
    predProba=clf.predict_proba(dist[['principal component 1', 'principal component 2', 'principal component 3' ]].to_numpy().reshape(-1, 3))
    dist['predicted_probability']=predProba[:,1]
    # dist['predicted_interaction']=clf.predict(dist[['principal component 1', 'principal component 2', 'principal component 3' ]].to_numpy().reshape(-1, 3))
    dist['predicted_interaction']= np.asarray(dist['predicted_probability']>=proba_theshold_prediction)
    list_predicted=dist.loc[dist['predicted_interaction']]['gene'].to_list()
    return (list_predicted, dist)

def get_id_from_name(goi):
    selec=(WBgene['geneName']==goi)
    if not any(selec):
        selec=(WBgene['sequence']==goi)
    name=WBgene.loc[selec]['geneId'].values[0]
    return name
    
def get_interaction(goi):
    name=get_id_from_name(goi)
    print(f"--- goi={goi} name={name} ----")
    json_file=os.path.join(cur_dir,"Data",f"gene_{name}_interactions.json")
    # Check if the file already exists
    if not os.path.exists(json_file):
        print("Downloading the file")
        # URL of the interaction file
        url = f"https://wormbase.org/rest/widget/gene/{name}/interactions?download=1&content-type=application%2Fjson"
        response = requests.get(url)
        # Save the file to the specified file path
        with open(json_file, 'wb') as json:
            json.write(response.content)
    return json_file
def get_list_from_goi(goi,tempe=temp,proba_theshold_prediction=proba_theshold_prediction):
    json_file=get_interaction(goi)
    (inter_klp19, pred_klp19)= find_interactor(goi,json_file,tempe=tempe,proba_theshold_prediction=proba_theshold_prediction)
    return (inter_klp19, pred_klp19)

def assemble_list(idx,common_genes,common_details,interactors):
    if idx == 0:
        common_genes = set(interactors)
        common_details = details.drop(columns=['predicted_probability'],inplace=False)
        common_details = common_details.filter(regex=r'^(?!dist_to)')
    else:
        common_genes = common_genes.intersection(interactors)
        common_details = pd.merge(common_details, details.drop(columns=['predicted_probability'],inplace=False).filter(regex=r'^(?!dist_to)'), on='gene', how='inner',suffixes=('', '_details'))        
        common_details['predicted_interaction'] = np.logical_and(common_details['predicted_interaction'], common_details['predicted_interaction_details'])
        common_details['interacting'] = np.logical_or(common_details['interacting'], common_details['interacting_details'])
        common_details = common_details.drop(columns=common_details.filter(regex='_details$').columns)
        display(common_details)
    return (common_genes,common_details)

def  output_formater(common_genes,common_details):
    display(common_genes)
    details_interacting_genes = common_details.loc[common_details['predicted_interaction']]
    count_known = len(details_interacting_genes[(details_interacting_genes['predicted_interaction'] == True) & (details_interacting_genes['interacting'] == True)])
    count_false_neg = len(details_interacting_genes[(details_interacting_genes['predicted_interaction'] == False) & (details_interacting_genes['interacting'] == True)])
    count_novel = len(details_interacting_genes[(details_interacting_genes['predicted_interaction'] == True) & (details_interacting_genes['interacting'] == False)])
    print(f"Number of known recovered interactions: {count_known}   Number of known but NOT recovered interactions: {count_false_neg} Number of novel interactions: {count_novel}   Total number of interactions: {count_known+count_novel}")
    details_interacting_genes[['principal component 1', 'principal component 2', 'principal component 3']] = details_interacting_genes[['principal component 1', 'principal component 2', 'principal component 3']].round(2)
    display(details_interacting_genes)
    return details_interacting_genes

## poleward flux after Steblyanko 2020

In [30]:
common_genes = set()
common_details = None
gois=['klp-19', 'bmk-1', 'klp-15', 'klp-16' , 'klp-17', 'klp-18', 'lin-5', 'klp-7']
for idx,goi in enumerate(gois):
    print(f"=============== interactors for {goi} ===============")
    interactors, details = get_list_from_goi(goi)
    print(f"--------------- found interactors ({goi}) --------------")
    pd.set_option('display.max_rows', 0) 
    display(details.loc[details['predicted_interaction']][['gene', 'interacting', 'predicted_interaction', 'predicted_probability']].sort_values('predicted_probability',ascending=False))
    pd.set_option('display.max_rows', 10) 
    common_genes,common_details = assemble_list(idx,common_genes,common_details,interactors)
print("=============== Common genes =================")
details_interacting_genes = output_formater(common_genes,common_details)

=============== interactors for klp-19 ===============
--- goi=klp-19 name=WBGene00002229 ----
---------- training set klp-19-------


,gene,principal component 1,principal component 2,principal component 3,interacting
10,L4440-18C,-0.723055,-1.546943,0.525031,False
11,L4440ebp2,-0.498394,0.984421,2.054312,False
50,klp19,-0.597349,-3.926979,-4.945162,True
63,none-18C,0.000000,0.000000,-0.000000,False
67,plk1-18C,0.424861,-8.712250,-6.001948,True
85,zen4,1.288270,8.995004,-10.353139,True


True
[LibLinear]iter  1 act 1.968e+00 pre 1.725e+00 delta 3.342e-01 f 4.159e+00 |g| 1.210e+01 CG   2
iter  2 act 3.795e-01 pre 3.110e-01 delta 4.143e-01 f 2.191e+00 |g| 3.063e+00 CG   3
iter  3 act 1.213e-01 pre 9.609e-02 delta 4.143e-01 f 1.811e+00 |g| 1.092e+00 CG   2
iter  4 act 3.436e-02 pre 3.135e-02 delta 4.143e-01 f 1.690e+00 |g| 3.740e-01 CG   2
iter  5 act 1.216e-03 pre 1.212e-03 delta 4.143e-01 f 1.656e+00 |g| 5.680e-02 CG   3
iter  6 act 7.229e-08 pre 7.228e-08 delta 4.143e-01 f 1.655e+00 |g| 7.037e-04 CG   2
On the training set: Accuracy: 1.0, Recall: 1.0, Precision: 1.0
--------------- found interactors (klp-19) --------------


,gene,interacting,predicted_interaction,predicted_probability
79,tpxl1-18C,False,True,0.999998
21,cls2-18C,False,True,0.999997
18,cdk1,False,True,0.999940
85,zen4,True,True,0.999873
40,gpb1,False,True,0.999726
83,unc59,False,True,0.999583
70,spd1,False,True,0.998568
74,such1dylt1,False,True,0.997916
14,air1,False,True,0.997597
...,...,...,...,...


=============== interactors for bmk-1 ===============
--- goi=bmk-1 name=WBGene00000257 ----
---------- training set bmk-1-------


...

=============== Common genes =================


{'air1', 'cdk1', 'tpxl1-18C'}

Number of known recovered interactions: 1   Number of known but NOT recovered interactions: 0 Number of novel interactions: 2   Total number of interactions: 3


/var/folders/rh/qywtqnw925n8ms_lmg7rmxn40000gn/T/ipykernel_27740/1044818270.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  details_interacting_genes[['principal component 1', 'principal component 2', 'principal component 3']] = details_interacting_genes[['principal component 1', 'principal component 2', 'principal component 3']].round(2)


,gene,principal component 1,principal component 2,principal component 3,interacting,predicted_interaction
14,air1,-4.21,1.17,-9.40,True,True
18,cdk1,-1.67,-5.07,-13.80,False,True
79,tpxl1-18C,-8.24,-3.02,-20.48,False,True


### poleward flux, Steblyanko except NCD (redundant) and bmk-1 (divergent function ?)

In [31]:
common_genes = set()
common_details = None
gois=['klp-19', 'klp-18', 'lin-5', 'klp-7']
for idx,goi in enumerate(gois):
    print(f"=============== interactors for {goi} ===============")
    interactors, details = get_list_from_goi(goi)
    print(f"--------------- found interactors ({goi}) --------------")
    pd.set_option('display.max_rows', 0) 
    display(details.loc[details['predicted_interaction']][['gene', 'interacting', 'predicted_interaction', 'predicted_probability']].sort_values('predicted_probability',ascending=False))
    pd.set_option('display.max_rows', 10) 
    common_genes,common_details = assemble_list(idx,common_genes,common_details,interactors)
print("=============== Common genes =================")
details_interacting_genes = output_formater(common_genes,common_details)

=============== interactors for klp-19 ===============
--- goi=klp-19 name=WBGene00002229 ----
---------- training set klp-19-------


,gene,principal component 1,principal component 2,principal component 3,interacting
10,L4440-18C,-0.723055,-1.546943,0.525031,False
11,L4440ebp2,-0.498394,0.984421,2.054312,False
50,klp19,-0.597349,-3.926979,-4.945162,True
63,none-18C,0.000000,0.000000,-0.000000,False
67,plk1-18C,0.424861,-8.712250,-6.001948,True
85,zen4,1.288270,8.995004,-10.353139,True


True
[LibLinear]iter  1 act 1.968e+00 pre 1.725e+00 delta 3.342e-01 f 4.159e+00 |g| 1.210e+01 CG   2
iter  2 act 3.795e-01 pre 3.110e-01 delta 4.143e-01 f 2.191e+00 |g| 3.063e+00 CG   3
iter  3 act 1.213e-01 pre 9.609e-02 delta 4.143e-01 f 1.811e+00 |g| 1.092e+00 CG   2
iter  4 act 3.436e-02 pre 3.135e-02 delta 4.143e-01 f 1.690e+00 |g| 3.740e-01 CG   2
iter  5 act 1.216e-03 pre 1.212e-03 delta 4.143e-01 f 1.656e+00 |g| 5.680e-02 CG   3
iter  6 act 7.229e-08 pre 7.228e-08 delta 4.143e-01 f 1.655e+00 |g| 7.037e-04 CG   2
On the training set: Accuracy: 1.0, Recall: 1.0, Precision: 1.0
--------------- found interactors (klp-19) --------------


,gene,interacting,predicted_interaction,predicted_probability
79,tpxl1-18C,False,True,0.999998
21,cls2-18C,False,True,0.999997
18,cdk1,False,True,0.999940
85,zen4,True,True,0.999873
40,gpb1,False,True,0.999726
83,unc59,False,True,0.999583
70,spd1,False,True,0.998568
74,such1dylt1,False,True,0.997916
14,air1,False,True,0.997597
...,...,...,...,...


=============== interactors for klp-18 ===============
--- goi=klp-18 name=WBGene00002228 ----
---------- training set klp-18-------


...

=============== Common genes =================


{'air1', 'cdk1', 'tpxl1-18C'}

Number of known recovered interactions: 1   Number of known but NOT recovered interactions: 0 Number of novel interactions: 2   Total number of interactions: 3


/var/folders/rh/qywtqnw925n8ms_lmg7rmxn40000gn/T/ipykernel_27740/1044818270.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  details_interacting_genes[['principal component 1', 'principal component 2', 'principal component 3']] = details_interacting_genes[['principal component 1', 'principal component 2', 'principal component 3']].round(2)


,gene,principal component 1,principal component 2,principal component 3,interacting,predicted_interaction
14,air1,-4.21,1.17,-9.40,True,True
18,cdk1,-1.67,-5.07,-13.80,False,True
79,tpxl1-18C,-8.24,-3.02,-20.48,False,True


## spindle length tpxl-1

In [32]:
gois=['tpxl-1']
common_genes = set()
common_details = None
for idx,goi in enumerate(gois):
    print(f"=============== interactors for {goi} ===============")
    interactors, details = get_list_from_goi(goi)
    print(f"--------------- found interactors ({goi}) --------------")
    pd.set_option('display.max_rows', 0) 
    display(details.loc[details['predicted_interaction']][['gene', 'interacting', 'predicted_interaction', 'predicted_probability']].sort_values('predicted_probability',ascending=False))
    pd.set_option('display.max_rows', 10) 
    common_genes,common_details = assemble_list(idx,common_genes,common_details,interactors)
print("=============== Common genes =================")
details_interacting_genes = output_formater(common_genes,common_details)

=============== interactors for tpxl-1 ===============
--- goi=tpxl-1 name=WBGene00021470 ----
---------- training set tpxl-1-------


,gene,principal component 1,principal component 2,principal component 3,interacting
10,L4440-18C,-0.723055,-1.546943,0.525031,False
11,L4440ebp2,-0.498394,0.984421,2.054312,False
14,air1,-4.206413,1.172099,-9.398000,True
63,none-18C,0.000000,0.000000,-0.000000,False
79,tpxl1-18C,-8.239139,-3.023608,-20.483315,True


True
[LibLinear]iter  1 act 1.141e+00 pre 9.811e-01 delta 1.365e-01 f 3.466e+00 |g| 1.719e+01 CG   1
cg reaches trust region boundary
iter  2 act 3.809e-01 pre 3.133e-01 delta 2.181e-01 f 2.324e+00 |g| 4.935e+00 CG   2
cg reaches trust region boundary
iter  3 act 2.589e-01 pre 2.208e-01 delta 3.470e-01 f 1.944e+00 |g| 2.042e+00 CG   2
iter  4 act 1.108e-01 pre 1.039e-01 delta 3.470e-01 f 1.685e+00 |g| 8.055e-01 CG   2
iter  5 act 4.723e-03 pre 4.405e-03 delta 3.470e-01 f 1.574e+00 |g| 1.579e-01 CG   1
iter  6 act 1.482e-04 pre 1.468e-04 delta 3.470e-01 f 1.569e+00 |g| 2.093e-02 CG   2
On the training set: Accuracy: 1.0, Recall: 1.0, Precision: 1.0
--------------- found interactors (tpxl-1) --------------


,gene,interacting,predicted_interaction,predicted_probability
21,cls2-18C,False,True,0.999989
79,tpxl1-18C,True,True,0.999966
85,zen4,False,True,0.999903
18,cdk1,False,True,0.999372
83,unc59,False,True,0.998939
70,spd1,False,True,0.998781
40,gpb1,False,True,0.997557
14,air1,True,True,0.994481
20,clip1-18C,False,True,0.987957
74,such1dylt1,False,True,0.985495


=============== Common genes =================


{'JEP5mbk2',
 'air1',
 'cdk1',
 'clip1-18C',
 'cls2-18C',
 'gpb1',
 'klp7',
 'par4',
 'spd1',
 'spd2-18C',
 'spn4',
 'such1L4440',
 'such1dylt1',
 'tpxl1-18C',
 'unc59',
 'zen4'}

Number of known recovered interactions: 2   Number of known but NOT recovered interactions: 0 Number of novel interactions: 14   Total number of interactions: 16


/var/folders/rh/qywtqnw925n8ms_lmg7rmxn40000gn/T/ipykernel_27740/1044818270.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  details_interacting_genes[['principal component 1', 'principal component 2', 'principal component 3']] = details_interacting_genes[['principal component 1', 'principal component 2', 'principal component 3']].round(2)


,gene,principal component 1,principal component 2,principal component 3,interacting,predicted_interaction
8,JEP5mbk2,-0.75,-0.44,-5.19,False,True
14,air1,-4.21,1.17,-9.40,True,True
18,cdk1,-1.67,-5.07,-13.80,False,True
20,clip1-18C,-0.92,1.17,-6.73,False,True
21,cls2-18C,-1.12,3.49,-16.64,False,True
...,...,...,...,...,...,...
73,such1L4440,0.42,-6.65,-7.32,False,True
74,such1dylt1,0.52,-6.24,-8.41,False,True
79,tpxl1-18C,-8.24,-3.02,-20.48,True,True
83,unc59,-0.18,1.24,-10.09,False,True


## spindle length after Greenan 2010

In [33]:
gois=['tpxl-1', 'air-1']
for idx,goi in enumerate(gois):
    print(f"=============== interactors for {goi} ===============")
    interactors, details = get_list_from_goi(goi)
    print(f"--------------- found interactors ({goi}) --------------")
    pd.set_option('display.max_rows', 0) 
    display(details.loc[details['predicted_interaction']][['gene', 'interacting', 'predicted_interaction', 'predicted_probability']].sort_values('predicted_probability',ascending=False))
    pd.set_option('display.max_rows', 10) 
    common_genes,common_details = assemble_list(idx,common_genes,common_details,interactors)
print("=============== Common genes =================")
details_interacting_genes = output_formater(common_genes,common_details)

=============== interactors for tpxl-1 ===============
--- goi=tpxl-1 name=WBGene00021470 ----
---------- training set tpxl-1-------


,gene,principal component 1,principal component 2,principal component 3,interacting
10,L4440-18C,-0.723055,-1.546943,0.525031,False
11,L4440ebp2,-0.498394,0.984421,2.054312,False
14,air1,-4.206413,1.172099,-9.398000,True
63,none-18C,0.000000,0.000000,-0.000000,False
79,tpxl1-18C,-8.239139,-3.023608,-20.483315,True


True
[LibLinear]iter  1 act 1.141e+00 pre 9.811e-01 delta 1.365e-01 f 3.466e+00 |g| 1.719e+01 CG   1
cg reaches trust region boundary
iter  2 act 3.809e-01 pre 3.133e-01 delta 2.181e-01 f 2.324e+00 |g| 4.935e+00 CG   2
cg reaches trust region boundary
iter  3 act 2.589e-01 pre 2.208e-01 delta 3.470e-01 f 1.944e+00 |g| 2.042e+00 CG   2
iter  4 act 1.108e-01 pre 1.039e-01 delta 3.470e-01 f 1.685e+00 |g| 8.055e-01 CG   2
iter  5 act 4.723e-03 pre 4.405e-03 delta 3.470e-01 f 1.574e+00 |g| 1.579e-01 CG   1
iter  6 act 1.482e-04 pre 1.468e-04 delta 3.470e-01 f 1.569e+00 |g| 2.093e-02 CG   2
On the training set: Accuracy: 1.0, Recall: 1.0, Precision: 1.0
--------------- found interactors (tpxl-1) --------------


,gene,interacting,predicted_interaction,predicted_probability
21,cls2-18C,False,True,0.999989
79,tpxl1-18C,True,True,0.999966
85,zen4,False,True,0.999903
18,cdk1,False,True,0.999372
83,unc59,False,True,0.998939
70,spd1,False,True,0.998781
40,gpb1,False,True,0.997557
14,air1,True,True,0.994481
20,clip1-18C,False,True,0.987957
74,such1dylt1,False,True,0.985495


=============== interactors for air-1 ===============
--- goi=air-1 name=WBGene00000098 ----
---------- training set air-1-------


...

=============== Common genes =================


{'JEP5mbk2',
 'air1',
 'cdk1',
 'clip1-18C',
 'cls2-18C',
 'gpb1',
 'klp7',
 'par4',
 'spd2-18C',
 'spn4',
 'such1L4440',
 'such1dylt1',
 'tpxl1-18C',
 'unc59'}

Number of known recovered interactions: 5   Number of known but NOT recovered interactions: 0 Number of novel interactions: 9   Total number of interactions: 14


/var/folders/rh/qywtqnw925n8ms_lmg7rmxn40000gn/T/ipykernel_27740/1044818270.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  details_interacting_genes[['principal component 1', 'principal component 2', 'principal component 3']] = details_interacting_genes[['principal component 1', 'principal component 2', 'principal component 3']].round(2)


,gene,principal component 1,principal component 2,principal component 3,interacting,predicted_interaction
8,JEP5mbk2,-0.75,-0.44,-5.19,True,True
14,air1,-4.21,1.17,-9.40,True,True
18,cdk1,-1.67,-5.07,-13.80,True,True
20,clip1-18C,-0.92,1.17,-6.73,False,True
21,cls2-18C,-1.12,3.49,-16.64,False,True
...,...,...,...,...,...,...
72,spn4,0.53,-2.64,-4.55,False,True
73,such1L4440,0.42,-6.65,-7.32,False,True
74,such1dylt1,0.52,-6.24,-8.41,False,True
79,tpxl1-18C,-8.24,-3.02,-20.48,True,True


In [34]:
gois=['C27D9.1', 'ani-2','ima-3', 'F21H12.2'] # cid-1 was acquired at 18C
for idx,goi in enumerate(gois):
    print(f"=============== interactors for {goi} ===============")
    interactors, details = get_list_from_goi(goi,tempe='23C',proba_theshold_prediction=0.5)
    print(f"--------------- found interactors ({goi}) --------------")
    pd.set_option('display.max_rows', 0) 
    display(details.loc[details['predicted_interaction']][['gene', 'interacting', 'predicted_interaction', 'predicted_probability']].sort_values('predicted_probability',ascending=False))
    pd.set_option('display.max_rows', 10) 
    if idx == 0:
        common_genes = set(interactors)
    else:
        common_genes = common_genes.intersection(interactors)
print("=============== Common genes =================")
display(common_genes)

=============== interactors for C27D9.1 ===============
--- goi=C27D9.1 name=WBGene00016163 ----
---------- training set C27D9.1-------


,gene,principal component 1,principal component 2,principal component 3,interacting


True


Exception: Something wrong in naming ? No interacting gene in our dataset

## spd-1 (central spindle)

In [ ]:
gois=['spd-1'] 
for idx,goi in enumerate(gois):
    print(f"=============== interactors for {goi} ===============")
    interactors, details = get_list_from_goi(goi,tempe='18C',proba_theshold_prediction=0.95)
    print(f"--------------- found interactors ({goi}) --------------")
    pd.set_option('display.max_rows', 0) 
    display(details.loc[details['predicted_interaction']][['gene', 'interacting', 'predicted_interaction', 'predicted_probability']].sort_values('predicted_probability',ascending=False))
    pd.set_option('display.max_rows', 10) 
    common_genes,common_details = assemble_list(idx,common_genes,common_details,interactors)
print("=============== Common genes =================")
details_interacting_genes = output_formater(common_genes,common_details)